In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import re
import joblib
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import FeatureHasher
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm

class DFToDictTransformer(BaseEstimator, TransformerMixin):
    """Custom transformer to convert DataFrame columns to dictionary format for feature hashing"""
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.columns].to_dict(orient='records')

def build_preprocessor():
    """Build preprocessing pipeline for numeric and categorical features"""
    numeric_features = ['ip_first_octet', 'ip_second_octet', 'isRootUser', 'has_suspicious_arn']
    categorical_features = ['eventName', 'eventSource', 'errorCode']

    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('to_dict', DFToDictTransformer(categorical_features)),
        ('hasher', FeatureHasher(n_features=256, input_type='dict'))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        sparse_threshold=0
    )
    return preprocessor

def preprocess_data(df):
    """Preprocess cloud security log data for model training"""
    df['sourceIPAddress'] = df['sourceIPAddress'].fillna('0.0.0.0')
    df['ip_first_octet'] = df['sourceIPAddress'].apply(lambda x: int(str(x).split('.')[0]) if str(x).split('.')[0].isdigit() else 0)
    df['ip_second_octet'] = df['sourceIPAddress'].apply(lambda x: int(str(x).split('.')[1]) if len(str(x).split('.')) > 1 and str(x).split('.')[1].isdigit() else 0)

    df['userIdentityarn'] = df['userIdentityarn'].fillna('')
    df['has_suspicious_arn'] = df['userIdentityarn'].apply(lambda x: 1 if re.search(r'admin|root|super', x, re.IGNORECASE) else 0)
    df['isRootUser'] = (df['userIdentitytype'] == 'Root').astype(int)
    df['errorCode'] = df['errorCode'].fillna('NoError')

    df = df.drop(columns=['eventID', 'userIdentitytype', 'userIdentityarn', 'sourceIPAddress'])
    return df

def train_autoencoder(X, hidden_layer_sizes=(30, 15, 30), max_iter=100):
    """Train autoencoder for anomaly detection"""
    if hasattr(X, 'toarray'):
        X = X.toarray()
    autoencoder = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes, max_iter=max_iter, random_state=42)
    autoencoder.fit(X, X)
    return autoencoder

def train_models(df):
    """Train multi-model ensemble for cloud security anomaly detection"""
    print("Preprocessing data...")
    df = preprocess_data(df)
    preprocessor = build_preprocessor()
    preprocessor.fit(df)
    
    X = preprocessor.transform(df)
    if hasattr(X, "toarray"):
        X = X.toarray()

    print("Training Isolation Forest with optimized contamination...")
    # Use low contamination for anomaly detection
    isolation_forest = IsolationForest(
        n_estimators=100, 
        contamination=0.001,  # Only 0.1% should be anomalous
        random_state=42,
        max_samples='auto'
    )
    isolation_forest.fit(X)

    # Get anomaly scores and find proper threshold
    iso_scores = isolation_forest.decision_function(X)
    iso_scores_flipped = -iso_scores
    
    # Use percentiles for different risk levels
    iso_threshold_99_9 = np.percentile(iso_scores_flipped, 99.9)
    iso_threshold_99 = np.percentile(iso_scores_flipped, 99.0)
    iso_threshold_95 = np.percentile(iso_scores_flipped, 95.0)
    
    print(f"99.9th percentile threshold: {iso_threshold_99_9:.4f}")
    print(f"99th percentile threshold: {iso_threshold_99:.4f}")
    print(f"95th percentile threshold: {iso_threshold_95:.4f}")
    
    # Save thresholds for different risk levels
    joblib.dump({
        'threshold_99_9': iso_threshold_99_9,
        'threshold_99': iso_threshold_99,
        'threshold_95': iso_threshold_95,
        'min_score': iso_scores_flipped.min(),
        'max_score': iso_scores_flipped.max()
    }, '/kaggle/working/iso_thresholds.joblib')

    print("Training Random Forest with balanced labels...")
    # Create more balanced labels - only top 0.1% are anomalies
    iso_preds = isolation_forest.predict(X)
    y = (iso_preds == -1).astype(int)
    
    print(f"Anomaly ratio in training: {y.mean():.4f}")
    
    # Use class weights to handle extreme imbalance
    n_anomalies = y.sum()
    n_normal = len(y) - n_anomalies
    class_weights = {0: 1, 1: n_normal / n_anomalies}
    
    random_forest = RandomForestClassifier(
        n_estimators=100, 
        random_state=42,
        class_weight=class_weights,
        max_depth=8  # Prevent overfitting
    )
    random_forest.fit(X, y)

    print("Training Autoencoder...")
    autoencoder = train_autoencoder(X, hidden_layer_sizes=(30, 15, 30), max_iter=100)

    print("Calculating Autoencoder thresholds...")
    reconstructions = autoencoder.predict(X)
    if reconstructions.ndim == 1:
        reconstructions = reconstructions.reshape(-1, 1)
    if reconstructions.shape[1] != X.shape[1]:
        reconstructions = np.tile(reconstructions, (1, X.shape[1]))

    errors = np.mean((X - reconstructions) ** 2, axis=1)
    
    # Use percentiles for autoencoder too
    ae_threshold_99_9 = np.percentile(errors, 99.9)
    ae_threshold_99 = np.percentile(errors, 99.0)
    ae_threshold_95 = np.percentile(errors, 95.0)
    
    print(f"AE 99.9th percentile threshold: {ae_threshold_99_9:.4f}")
    print(f"AE 99th percentile threshold: {ae_threshold_99:.4f}")
    print(f"AE 95th percentile threshold: {ae_threshold_95:.4f}")
    
    joblib.dump({
        'threshold_99_9': ae_threshold_99_9,
        'threshold_99': ae_threshold_99,
        'threshold_95': ae_threshold_95,
        'mean_error': np.mean(errors),
        'std_error': np.std(errors)
    }, '/kaggle/working/ae_thresholds.joblib')

    # Save models
    joblib.dump(preprocessor, '/kaggle/working/preprocessor.joblib')
    joblib.dump(isolation_forest, '/kaggle/working/isolation_forest.joblib')
    joblib.dump(random_forest, '/kaggle/working/random_forest.joblib')
    joblib.dump(autoencoder, '/kaggle/working/autoencoder.joblib')

    print("Models and thresholds saved.")

def score_single_log(log_dict):
    """Score individual log entries using trained ensemble models"""
    preprocessor = joblib.load('/kaggle/working/preprocessor.joblib')
    isolation_forest = joblib.load('/kaggle/working/isolation_forest.joblib')
    random_forest = joblib.load('/kaggle/working/random_forest.joblib')
    autoencoder = joblib.load('/kaggle/working/autoencoder.joblib')
    iso_thresholds = joblib.load('/kaggle/working/iso_thresholds.joblib')
    ae_thresholds = joblib.load('/kaggle/working/ae_thresholds.joblib')

    df_log = pd.DataFrame([log_dict])
    
    # Preprocessing
    df_log['eventVersion'] = pd.to_numeric(df_log['eventVersion'], errors='coerce').fillna(0)
    df_log['userIdentityaccountId'] = pd.to_numeric(df_log['userIdentityaccountId'], errors='coerce').fillna(0)
    df_log['sourceIPAddress'] = df_log['sourceIPAddress'].astype(str)
    df_log = preprocess_data(df_log)
    
    X_log = preprocessor.transform(df_log)
    if hasattr(X_log, 'toarray'):
        X_log = X_log.toarray()

    # 1. Isolation Forest - Use percentile-based scoring
    iso_pred_raw = isolation_forest.decision_function(X_log)[0]
    iso_score = -iso_pred_raw
    
    # Convert to risk score based on thresholds
    if iso_score >= iso_thresholds['threshold_99_9']:
        iso_risk = 100  # Critical
    elif iso_score >= iso_thresholds['threshold_99']:
        iso_risk = 80   # High
    elif iso_score >= iso_thresholds['threshold_95']:
        iso_risk = 40   # Medium
    else:
        # Linear interpolation for low scores
        normalized = (iso_score - iso_thresholds['min_score']) / (iso_thresholds['threshold_95'] - iso_thresholds['min_score'])
        iso_risk = max(0, normalized * 30)  # 0-30 range for normal
    
    # 2. Random Forest - Use calibrated probabilities
    rf_pred_proba = random_forest.predict_proba(X_log)[0][1]
    # Convert to risk score (0-100)
    rf_risk = rf_pred_proba * 100
    
    # 3. Autoencoder - Use threshold-based scoring
    recon = autoencoder.predict(X_log)
    if recon.ndim == 1:
        recon = recon.reshape(1, -1)
    if recon.shape[1] != X_log.shape[1]:
        recon = np.tile(recon, (1, X_log.shape[1]))
    
    ae_error = np.mean((X_log - recon) ** 2)
    
    # Convert to risk score based on thresholds
    if ae_error >= ae_thresholds['threshold_99_9']:
        ae_risk = 100  # Critical
    elif ae_error >= ae_thresholds['threshold_99']:
        ae_risk = 80   # High
    elif ae_error >= ae_thresholds['threshold_95']:
        ae_risk = 40   # Medium
    else:
        # Linear interpolation for low scores
        normalized = (ae_error - ae_thresholds['mean_error']) / (ae_thresholds['threshold_95'] - ae_thresholds['mean_error'])
        ae_risk = max(0, normalized * 30)  # 0-30 range for normal
    
    # 4. Combine scores with ensemble weighting
    # Give more weight to isolation forest (most reliable for anomalies)
    final_score = 0.5 * iso_risk + 0.3 * rf_risk + 0.2 * ae_risk
    
    # 5. Apply risk transformation
    # Most logs should be low risk
    if final_score < 20:
        final_score = final_score * 0.5  # Reduce low scores further
    elif final_score > 80:
        final_score = final_score * 1.2  # Increase high scores slightly
    
    # 6. Round to nearest integer
    rounded_score = round(final_score)
    
    return rounded_score

def test_distribution(df, sample_size=1000):
    """Test the score distribution on a sample of logs"""
    print(f"Testing distribution on {sample_size} random logs...")
    
    test_logs = df.sample(n=sample_size, random_state=42)
    scores = []
    
    for idx, log in test_logs.iterrows():
        try:
            score = score_single_log(log.to_dict())
            scores.append(score)
        except Exception as e:
            print(f"Error scoring log {idx}: {e}")
            continue
    
    # Print distribution
    print("\n=== SCORE DISTRIBUTION ===")
    print(f"0-20: {sum(1 for s in scores if s <= 20)} ({sum(1 for s in scores if s <= 20)/len(scores)*100:.1f}%)")
    print(f"21-40: {sum(1 for s in scores if 21 <= s <= 40)} ({sum(1 for s in scores if 21 <= s <= 40)/len(scores)*100:.1f}%)")
    print(f"41-60: {sum(1 for s in scores if 41 <= s <= 60)} ({sum(1 for s in scores if 41 <= s <= 60)/len(scores)*100:.1f}%)")
    print(f"61-80: {sum(1 for s in scores if 61 <= s <= 80)} ({sum(1 for s in scores if 61 <= s <= 80)/len(scores)*100:.1f}%)")
    print(f"81-100: {sum(1 for s in scores if s >= 81)} ({sum(1 for s in scores if s >= 81)/len(scores)*100:.1f}%)")
    
    print(f"\nMean score: {np.mean(scores):.2f}")
    print(f"Median score: {np.median(scores):.2f}")
    print(f"Std score: {np.std(scores):.2f}")
    
    return scores

# Main execution
if __name__ == "__main__":
    print("=== STARTING MODEL TRAINING ===")
    
    # Load data
    print("Loading data...")
    df_full = pd.read_csv('/kaggle/input/aws-cloudtrails-dataset-from-flaws-cloud/dec12_18features.csv')
    df = df_full.sample(n=100000, random_state=42).reset_index(drop=True)
    print(f"Loaded {len(df)} samples")
    
    # Train models with optimized parameters
    print("\n=== TRAINING MODELS ===")
    train_models(df.copy())
    
    # Test distribution
    print("\n=== TESTING DISTRIBUTION ===")
    scores = test_distribution(df, sample_size=1000)
    
    # Test a few specific examples
    print("\n=== TESTING SPECIFIC EXAMPLES ===")
    example_logs = df.sample(n=5, random_state=123)
    
    for idx, log in example_logs.iterrows():
        try:
            score = score_single_log(log.to_dict())
            event_name = log.get('eventName', 'Unknown')
            user_type = log.get('userIdentitytype', 'Unknown')
            print(f"Log {idx}: {event_name} by {user_type} -> Score: {score}")
        except Exception as e:
            print(f"Error with log {idx}: {e}")
    
    print("\n=== TRAINING COMPLETE ===")
    print("Models saved to /kaggle/working/")

In [ ]:
def parse_log_string(log_string):
    """
    Convert a pipe-separated log string to dictionary format for scoring
    
    Expected format: feature1|feature2|feature3|...|feature18
    """
    columns = [
        "eventID", "eventTime", "sourceIPAddress", "userAgent", "eventName",
        "eventSource", "awsRegion", "eventVersion", "userIdentitytype",
        "eventType", "userIdentityaccountId", "userIdentityprincipalId",
        "userIdentityarn", "userIdentityaccessKeyId", "userIdentityuserName",
        "errorCode", "errorMessage", "requestParametersinstanceType"
    ]
    
    # Split the string by pipe
    values = log_string.strip().split('|')
    
    # Check if we have the right number of features
    if len(values) != len(columns):
        raise ValueError(f"Expected {len(columns)} features, got {len(values)}")
    
    # Create dictionary
    log_dict = dict(zip(columns, values))
    
    return log_dict

def score_log_string(log_string):
    """
    Score a pipe-separated log string directly
    """
    log_dict = parse_log_string(log_string)
    return score_single_log_corrected(log_dict)

# Example usage:
if __name__ == "__main__":
    log_string = "9e6fb24d-e84d-40dd-8975-83a4f5c7f902|2018-07|253.6|aws-cli|DescribeSnapshots|ec2.amazonaws.com|us-west-2|1.05|IAMUser|AwsApiCall|811596193553.0|AIDA9BO36HFBHKGJAO9C1|arn:aws:iam::811596193553:user/backup|AKIA01U43UX3RBRDXF4Q|backup|NaN|NoError|NotApplicable"

    score = score_log_string(log_string)
    print(f"Risk score: {score}")